<a href="https://colab.research.google.com/github/zyz314/CS-7641-assignments_1/blob/master/Copy_of_Untitled40.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

课程名称: CS7641 Machine Learning
作业编号: Assignment 1
学期: Spring 2025
作者: [您的姓名]
日期: [提交日期]


2. 引言（Introduction）
2.1 研究背景
监督学习（Supervised Learning）在分类问题中的广泛应用。
选择 两个不同的数据集 来研究 不同监督学习算法的表现：
Customer Personality Dataset（客户个性数据集） → 预测客户是否属于高消费群体。
Spotify 2023 Dataset（Spotify 2023 数据集） → 预测歌曲是否会成为热门歌曲。

2.2 研究目标
本实验的目标：

探索两种数据集的特性：
数据是否平衡？
是否存在缺失值？
哪些特征对目标变量有影响？

比较四种分类算法的表现：
Neural Networks（神经网络）
Support Vector Machine（SVM，支持向量机）
k-Nearest Neighbors（k-NN，k-最近邻）
AdaBoost（Boosting for Decision Trees，决策树提升算法）

分析不同超参数对模型性能的影响：
训练误差 vs. 测试误差（学习曲线）。
过拟合 vs. 欠拟合（模型复杂度曲线）。
训练时间 & 预测时间对比。

2.3 实验方法
先对数据集进行探索与预处理，确保数据可以用于机器学习模型。
先完成 Customer Dataset 的所有算法实验，然后再完成 Spotify Dataset 的实验。
最后对比两种数据集上不同算法的表现。

3. 数据集 1：Customer Personality Dataset

3.1 数据加载
从 Kaggle 加载数据，展示前 5 行样本。
数据维度（行数、列数）。
目标变量 income 分布（>50K vs <=50K，是否类别不均衡？）。
📌 生成的图片：

customer_dataset_head.png → 数据集前 5 行展示。
customer_income_distribution.png → income 目标变量的类别分布饼图。

In [ ]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt

# Step 1: Mount Google Drive (if loading dataset from Google Drive)
# from google.colab import drive
# drive.mount('/content/drive')

# Step 2: Load the dataset (Assume file is uploaded to Colab)
file_path = "/content/marketing_campaign.csv"  # Modify the path if needed
customer_df = pd.read_csv(file_path, delimiter=";")  # Ensure the correct delimiter (semicolon in this case)

# Step 3: Display dataset dimensions (rows & columns)
num_rows, num_cols = customer_df.shape
print(f"Dataset Shape: {num_rows} rows, {num_cols} columns")

# Step 4: Display the first 5 rows of the dataset
print("First 5 rows of the dataset:")
print(customer_df.head())

# Step 5: Save the first 5 rows as a CSV file (optional, for report use)
customer_df.head().to_csv("/content/customer_dataset_head.csv", index=False)

# Step 6: Check if the dataset contains the 'Income' column
if "Income" in customer_df.columns:
    # Step 6.1: Convert 'Income' to numeric (in case it contains non-numeric values)
    customer_df['Income'] = pd.to_numeric(customer_df['Income'], errors='coerce')

    # Step 6.2: Define income classification threshold (50K USD)
    threshold = 50000

    # Step 6.3: Create a new binary target variable 'Income_Label'
    customer_df['Income_Label'] = customer_df['Income'].apply(lambda x: '>50K' if x > threshold else '<=50K')

    # Step 6.4: Count occurrences of each income category
    income_counts = customer_df['Income_Label'].value_counts()

    # Step 6.5: Plot the income distribution as a pie chart
    plt.figure(figsize=(6, 6))  # Set figure size
    plt.pie(income_counts, labels=income_counts.index, autopct='%1.1f%%',
            colors=["lightblue", "orange"], startangle=140)
    plt.title("Income Distribution in Customer Dataset")  # Add title

    # Step 6.6: Save the pie chart as an image file
    plt.savefig("/content/customer_income_distribution.png")

    # Step 6.7: Display the pie chart
    plt.show()

else:
    print("Warning: 'Income' column not found in dataset. Please verify the dataset structure.")


Dataset Shape: 2240 rows, 1 columns
First 5 rows of the dataset:
  ID\tYear_Birth\tEducation\tMarital_Status\tIncome\tKidhome\tTeenhome\tDt_Customer\tRecency\tMntWines\tMntFruits\tMntMeatProducts\tMntFishProducts\tMntSweetProducts\tMntGoldProds\tNumDealsPurchases\tNumWebPurchases\tNumCatalogPurchases\tNumStorePurchases\tNumWebVisitsMonth\tAcceptedCmp3\tAcceptedCmp4\tAcceptedCmp5\tAcceptedCmp1\tAcceptedCmp2\tComplain\tZ_CostContact\tZ_Revenue\tResponse
0  5524\t1957\tGraduation\tSingle\t58138\t0\t0\t0...                                                                                                                                                                                                                                                                                                                                                  
1  2174\t1954\tGraduation\tSingle\t46344\t1\t1\t0...                                                                                                     

3.2 数据探索
3.2.1 数值特征的分布
age, education-num, capital-gain, capital-loss, hours-per-week。
直方图展示数值变量分布。

📌 生成的图片： 3. customer_numeric_histograms.png → 所有数值特征的分布直方图。

生成的图片： 5. customer_missing_values_before.png → 缺失值统计（填充前）。 6. customer_missing_values_after.png → 缺失值统计（填充后）。

3.3 数据预处理
3.3.1 缺失值处理
数值特征 → 用均值填充。
类别特征 → 用众数填充或 "Unknown" **填充。**

3.2.2 类别特征的统计
workclass, education, marital-status, occupation, relationship, race, sex。
使用 条形图 观察类别变量分布。

3.3.3 类别特征编码
One-Hot 编码类别变量（workclass, education, marital-status 等）。
📌 生成的图片： 9. customer_ohe_features.png → One-Hot 编码后的特征数量统计。

3.4 训练集 & 测试集划分
80% 训练集，20% 测试集（使用 Stratified Sampling 保持类别比例）。
显示划分后的数据统计信息。
📌 生成的图片： 10. customer_train_test_split.png → 训练集 vs 测试集的目标变量分布。

3.5 Customer Dataset 的分类实验
Neural Networks（神经网络）
SVM（支持向量机）
k-NN（k-最近邻）
AdaBoost（Boosting for Decision Trees）
📌 生成的图片： 10. customer_dataset_roc_curve.png → Customer Dataset 上四种算法的 ROC 曲线（包含 AUC 值）。

4. 数据集 2：Spotify 2023 Dataset
📌 与 Customer Dataset 相同的分析流程。

📌 生成的图片： 11. spotify_dataset_roc_curve.png → Spotify Dataset 上四种算法的 ROC 曲线（包含 AUC 值）。




4.1 数据加载
从 Kaggle 加载数据，展示前 5 行样本。
数据维度（行数、列数）。
目标变量 profit_label 分布（0 代表非热门歌曲，1 代表热门歌曲）。
📌 生成的图片：

spotify_dataset_head.png → 数据集前 5 行展示。
spotify_profit_label_distribution.png → profit_label 目标变量的类别分布饼图。

4.2 数据探索
4.2.1 数值特征的分布
主要数值特征：

popularity（歌曲流行度）
budget（制作成本）
runtime（时长）
vote_count（投票数量）
vote_average（平均评分）
release_week（发行周）
release_month（发行月份）
title_len（歌曲标题长度）
直方图展示数值变量的分布情况。

📌 生成的图片： 3. spotify_numeric_histograms.png → 所有数值特征的分布直方图。

4.2.2 类别特征的统计
主要类别特征：

genres（音乐类型）
release_day（发行日期）
has_homepage（是否有主页）
cast_size（音乐制作团队大小）
使用条形图观察类别变量的分布。

📌 生成的图片： 4. spotify_categorical_barplots.png → 类别变量分布条形图。


4.3 数据预处理
4.3.1 缺失值处理
数值特征 → 用均值填充。
类别特征 → 用众数填充或 "Unknown" 填充。
📌 生成的图片： 5. spotify_missing_values_before.png → 缺失值统计（填充前）。 6. spotify_missing_values_after.png → 缺失值统计（填充后）。




4.3.2 特征变换
对 budget 和 revenue 进行对数变换 以减少偏态影响。
归一化数值变量（Min-Max 归一化）。
📌 生成的图片： 7. spotify_budget_log_transform.png → budget 对数变换前后对比。 8. spotify_revenue_log_transform.png → revenue 对数变换前后对比。



4.3.3 类别特征编码
One-Hot 编码类别变量（genres, release_day 等）。
📌 生成的图片： 9. spotify_ohe_features.png → One-Hot 编码后的特征数量统计。

4.4 训练集 & 测试集划分
80% 训练集，20% 测试集（使用 Stratified Sampling 保持类别比例）。
显示划分后的数据统计信息。
📌 生成的图片： 10. spotify_train_test_split.png → 训练集 vs 测试集的目标变量分布。

4.5 Spotify Dataset 的分类实验
（使用相同的四种分类算法进行实验）

4.5.1 Neural Networks（神经网络）
测试两种激活函数（ReLU 和 Sigmoid）。
调整隐藏层大小（100, [100,50], [100,50,25]）。
评估：准确率、ROC 曲线、训练时间。

4.5.2 SVM（支持向量机）
测试不同核函数（linear vs rbf）。
调整 C 值（1.0, 10.0, 100.0）。
评估：准确率、ROC 曲线、训练时间。

4.5.3 k-NN（k-最近邻）
测试不同 k 值（3, 5, 10, 20, 50）。
尝试不同的距离度量（欧几里得 vs 曼哈顿）。
评估：准确率、ROC 曲线、训练时间。

4.5.4 AdaBoost（Boosting for Decision Trees）
调整 n_estimators（50, 100, 200）和 learning_rate（0.1, 0.5, 1.0）。
评估：准确率、ROC 曲线、训练时间。
📌 生成的图片： 11. spotify_dataset_roc_curve.png → Spotify Dataset 上四种算法的 ROC 曲线（包含 AUC 值）。

4.6 Spotify Dataset 结果分析
4.6.1 不同算法的准确率对比
评估不同算法的分类准确率，选择最佳模型。
📌 生成的图片： 12. spotify_accuracy_comparison.png → 不同算法在 Spotify 数据集上的准确率对比。

4.6.2 训练时间 & 预测时间对比
分析计算时间，选择效率最高的算法。
📌 生成的图片： 13. spotify_training_time_comparison.png → 训练时间对比。 14. spotify_prediction_time_comparison.png → 预测时间对比。



4.6.3 模型复杂度曲线
不同超参数（C, k, n_estimators）如何影响误差？
📌 生成的图片： 15. spotify_model_complexity_svm.png → SVM 在不同 C 值下的模型复杂度曲线。 16. spotify_model_complexity_knn.png → k-NN 在不同 k 值下的模型复杂度曲线。 17. spotify_model_complexity_adaboost.png → AdaBoost 在不同 n_estimators 下的模型复杂度曲线。

5. 结果对比
📌 与 Customer Dataset 进行全面对比：

哪个数据集对不同算法更友好？
类别不均衡 vs. 平衡数据集，对模型影响如何？
特征的不同如何影响分类任务？
📌 生成的图片： 18. dataset_comparison.png → 数据集特征对比可视化。 19. comparison_accuracy.png → 所有算法的准确率比较。 20. comparison_training_time.png → 训练时间比较。 21. comparison_testing_time.png → 预测时间比较。

6. 结论与未来工作
📌 总结最佳算法，并讨论未来优化方向。

📌 生成的图片： 22. summary_best_model.png → 最佳模型总结图。